## Trump Model Training

In [1]:
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras import regularizers
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
## define paths to data files
path_to_train_data = Path("C:/Users/gutda/pycharm_projects/jass-data/filtered_to_csv/train/")
path_to_test_data = Path("C:/Users/gutda/pycharm_projects/jass-data/filtered_to_csv/test/")

In [3]:
# train data
data_train = pd.read_csv(path_to_train_data / 'filtered_data0001.csv', header=None)

In [4]:
# test data
data_test = pd.read_csv(path_to_test_data / 'filtered_data0001.csv', header=None)

In [6]:
# check how much of each trump action is there
data_train_f0 = (data_train[data_train[37]==0])
data_train_f1 = (data_train[data_train[37]==1])
data_train_f2 = (data_train[data_train[37]==2])
data_train_f3 = (data_train[data_train[37]==3])
data_train_f4 = (data_train[data_train[37]==4])
data_train_f5 = (data_train[data_train[37]==5])
data_train_f6 = (data_train[data_train[37]==6])

print(f"0 = {len(data_train_f0)}, 1 = {len(data_train_f1)}, 2 = {len(data_train_f2)}, 3 = {len(data_train_f3)}, 4 = {len(data_train_f4)}, 5 = {len(data_train_f5)}, 6 = {len(data_train_f6)}")


0 = 11999, 1 = 12325, 2 = 16495, 3 = 16178, 4 = 14856, 5 = 18270, 6 = 55951


In [9]:
# make sure every action has an equal number of samples, ignore action schiebe
data_train_f0 = data_train_f0[0: 11000]
data_train_f1 = data_train_f1[0: 11000]
data_train_f2 = data_train_f2[0: 11000]
data_train_f3 = data_train_f3[0: 11000]
data_train_f4 = data_train_f4[0: 11000]
data_train_f5 = data_train_f5[0: 11000]
data_train_f6 = data_train_f6[0: 2000] 

data_train = pd.concat([data_train_f0, data_train_f1, data_train_f2, data_train_f3, data_train_f4, data_train_f5, data_train_f6])
data_train = data_train.sample(frac=1)
data_train.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
2824,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,6
31658,0,0,0,0,0,1,0,0,1,0,...,0,0,1,1,1,1,1,0,0,3
16162,1,0,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,1,0,3
127187,1,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,0,1,0,1
114018,1,0,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [10]:
# check how much of each trump action is there
data_test_f0 = data_test[data_test[37]==0]
data_test_f1 = data_test[data_test[37]==1]
data_test_f2 = data_test[data_test[37]==2]
data_test_f3 = data_test[data_test[37]==3]
data_test_f4 = data_test[data_test[37]==4]
data_test_f5 = data_test[data_test[37]==5]
data_test_f6 = data_test[data_test[37]==6]

print(f"0 = {len(data_test_f0)}, 1 = {len(data_test_f1)}, 2 = {len(data_test_f2)}, 3 = {len(data_test_f3)}, 4 = {len(data_test_f4)}, 5 = {len(data_test_f5)}, 6 = {len(data_test_f6)}")


0 = 3906, 1 = 4096, 2 = 5459, 3 = 5373, 4 = 4830, 5 = 6093, 6 = 18647


In [62]:
# make sure every action has an equal number of samples, ignore action schiebe
data_test_f0 = data_test_f0[0:2200]
data_test_f1 = data_test_f1[0:2000]
data_test_f2 = data_test_f2[0:2000]
data_test_f3 = data_test_f3[0:2000]
data_test_f4 = data_test_f4[0:2000]
data_test_f5 = data_test_f5[0:2000]
data_test_f6 = data_test_f6[0:200] 

data_test = pd.concat([data_test_f0, data_test_f1, data_test_f2, data_test_f3, data_test_f4, data_test_f5, data_test_f6])
data_test = data_test.sample(frac=1)
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
8449,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,2
114,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
3181,0,0,1,1,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
10550,0,1,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,5
77,0,0,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,6


## Label data for readability

In [63]:
# Add colums. Each card and the information 
# if the player played forehand or not is a feature. That means we have 37 features.
# We also remove the player_id, because we do not care in the beginning

cards = [
# Diamonds
'DA','DK','DQ','DJ','D10','D9','D8','D7','D6',
# Hearts
'HA','HK','HQ','HJ','H10','H9','H8','H7','H6',
# Spades
'SA','SK','SQ','SJ','S10','S9','S8','S7','S6',
# Clubs
'CA','CK','CQ','CJ','C10','C9','C8','C7','C6'
]

# Forehand (yes = 1, no = 0)
forehand = ['FH']
trump = ['trump']

data_train.columns = cards + forehand + trump
data_test.columns = cards + forehand  + trump

data_train.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,CK,CQ,CJ,C10,C9,C8,C7,C6,FH,trump
2824,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,6
31658,0,0,0,0,0,1,0,0,1,0,...,0,0,1,1,1,1,1,0,0,3
16162,1,0,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,1,0,3
127187,1,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,0,1,0,1
114018,1,0,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


## Prepare x and y values

In [64]:
# colums used for extracting x and y values. The same effect could be achieved with train_test_split-Method, but 
# since we already have different files, we dont need to split the files using this method.
data_X_columns = cards + forehand
data_Y_colums = trump

x_train = data_train[data_X_columns]
y_train = data_train[data_Y_colums]

x_test = data_test[data_X_columns]
y_test = data_test[data_Y_colums]

## Create model and train it

In [67]:
# We have 37 features, so we have a node for each feature. There are 7 output categories: each trump color(4), 
# obe-abe, unde-ufe, schiebe. So we need an reducing function with 7 elements
model = keras.Sequential()
model.add(keras.layers.Dense(37, activation='relu', input_shape=[37]))
model.add(keras.layers.Dense(37, activation='relu'))
model.add(keras.layers.Dense(37, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model.add(keras.layers.Dense(37, activation='relu'))
model.add(keras.layers.Dense(7, activation='softmax'))

opt = Adam()

model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 37)                1406      
_________________________________________________________________
dense_6 (Dense)              (None, 37)                1406      
_________________________________________________________________
dense_7 (Dense)              (None, 37)                1406      
_________________________________________________________________
dense_8 (Dense)              (None, 37)                1406      
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 266       
Total params: 5,890
Trainable params: 5,890
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
# we need to convert the y_train to a array with 7 elements, to represent each 
# of the 7 categories
y_train_categorical = to_categorical(y_train)

history = model.fit(x_train, y_train_categorical, epochs=100, batch_size=100)

Train on 68000 samples
Epoch 1/100
68000/68000 [==============================] - 1s 13us/sample - loss: 1.0634 - accuracy: 0.6822
Epoch 2/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.7216 - accuracy: 0.7685
Epoch 3/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.6606 - accuracy: 0.7791
Epoch 4/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.6265 - accuracy: 0.7870
Epoch 5/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.6075 - accuracy: 0.7911
Epoch 6/100
68000/68000 [==============================] - 1s 13us/sample - loss: 0.5934 - accuracy: 0.7955
Epoch 7/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5858 - accuracy: 0.7974
Epoch 8/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5782 - accuracy: 0.7995
Epoch 9/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5735 - accuracy: 0.7992
Epoch

68000/68000 [==============================] - 1s 12us/sample - loss: 0.5213 - accuracy: 0.8160
Epoch 75/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5210 - accuracy: 0.8153
Epoch 76/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5216 - accuracy: 0.8156
Epoch 77/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5201 - accuracy: 0.8166
Epoch 78/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5206 - accuracy: 0.8161
Epoch 79/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5201 - accuracy: 0.8151
Epoch 80/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5196 - accuracy: 0.8169
Epoch 81/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5206 - accuracy: 0.8156
Epoch 82/100
68000/68000 [==============================] - 1s 12us/sample - loss: 0.5191 - accuracy: 0.8160
Epoch 83/100
68000/68000 [======

## Generate graphs for loss and accuracy

In [69]:
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.plot(history.history['accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['Train'], loc='upper left')

## Test model

In [70]:
y_test_categorical = to_categorical(y_test)
model.evaluate(x_test, y_test_categorical)

12300/12300 [==============================] - 0s 16us/sample - loss: 0.5139 - accuracy: 0.8159


[0.5138570079473945, 0.81585366]

In [71]:
model.save("final_model_81_games_025_mean_03_std_06_with_schieben.h5")